# Projeto Marinha do Brasil

## Laboratório de Processamento de Sinais - UFRJ

### Autor: Vinícius dos Santos Mello <viniciusdsmello@poli.ufrj.br>

In [1]:
import os
import sys
import time
sys.path.insert(0,'..')

from noveltyDetectionConfig import CONFIG
import numpy as np

import multiprocessing

from SAENoveltyDetectionAnalysis import SAENoveltyDetectionAnalysis

num_processes = multiprocessing.cpu_count()

analysis_name = 'StackedAutoEncoder'

# Enviroment variables
data_path = CONFIG['OUTPUTDATAPATH']
results_path = CONFIG['PACKAGE_NAME']

analysis = SAENoveltyDetectionAnalysis(analysis_name="StackedAutoEncoder", development_flag = False, development_events=400, loadData=True)
all_data, all_trgt, trgt_sparse = analysis.getData()

Using TensorFlow backend.


[+] Time to read data file: 16.0179388523 seconds
Qtd event of A is 12939
Qtd event of B is 29352
Qtd event of C is 11510
Qtd event of D is 23760

Biggest class is B with 29352 events
DataHandler Class: CreateEventsForClass
Original Size: (12939, 400)
DataHandler Class: CreateEventsForClass
Original Size: (29352, 400)
DataHandler Class: CreateEventsForClass
Original Size: (11510, 400)
DataHandler Class: CreateEventsForClass
Original Size: (23760, 400)


#### Load the train parameters file

In [2]:
analysis.setTrainParameters(n_inits=2,
                        hidden_activation='tanh',
                        output_activation='linear',
                        n_epochs=300,
                        n_folds=10,
                        patience=30,
                        batch_size=256,
                        verbose=False,
                        norm='without',
                        optmizerAlgorithm='Adam',
                        metrics=['accuracy'],
                        loss='mean_squared_error')

print ("\nResults path: " + analysis.getBaseResultsPath())

# trn_params = analysis.getTrainParameters()

Reading from /home/vinicius.mello/Workspace/SonarAnalysis/Results/NoveltyDetection/10_folds_cross_validation.jbl

Results path: /home/vinicius.mello/Workspace/SonarAnalysis/Results/NoveltyDetection/StackedAutoEncoder/Adam_optmizer/tanh_sae_hidden_activation/linear_sae_output_actvation/softmax_classifier_output_activation/2_init_10_folds_without_norm_300_epochs_256_batch_size_accuracy_metric_mean_squared_error_loss


#### Initialize objects of StackedAutoEncoder class for all models with its corresponding novelty class

In [3]:
analysis.createSAEModels()
SAE = analysis.getSAEModels()

#### Perform the training of the model

In [ ]:
for inovelty in range(1):
    startTime = time.time()
    analysis.train(layer=1,
                   inovelty=inovelty,
                   fineTuning=True,
                   trainingType="foldSweep", #foldSweep, neuronSweep, normal
                   hidden_neurons=[400],
                   neurons_variation_step=25,
                   numThreads=4,
                   regularizer=None,
                   regularizer_param=0)
    print "The training of the model for novelty {0} took {1} seconds to be performed\n".format(analysis.class_labels[inovelty], time.time() - startTime)

# Pre-training analysis

### Regularization impact

In [ ]:
# Neuron variation x KL Divergence

from sklearn import metrics

%matplotlib inline 

# Choose layer 
layer = 1

# Choose neurons topology
hidden_neurons = range(400,0,-50) + [2]

neurons_mat = [1] + range(50, 425, 25)
neurons_mat = neurons_mat[:len(neurons_mat)-layer+2]

# generate analysis data

verbose = True

regularizers = ['l1', 'l2']
regularizer_params = [10**i for i in range(-1,3)]

results = {"l1":{},
           "l2":{}
          }

for regularizer in regularizers:
    for regularizer_param in regularizer_params:
        results[regularizer][regularizer_param] = {}
        
        tempAnalysis = SAENoveltyDetectionAnalysis(analysis_name="StackedAutoEncoder", development_flag = False, development_events=400, loadData=False)
        tempAnalysis.all_data = all_data
        tempAnalysis.all_trgt = all_trgt
        tempAnalysis.trgt_sparse = trgt_sparse
        tempAnalysis.class_labels = analysis.class_labels
        
        tempAnalysis.setTrainParameters(n_inits=2,
                                        hidden_activation='tanh',
                                        output_activation='linear',
                                        n_epochs=300,
                                        n_folds=10,
                                        patience=30,
                                        batch_size=256,
                                        verbose=False,
                                        norm='without',
                                        optmizerAlgorithm='Adam',
                                        metrics=['accuracy'],
                                        loss='mean_squared_error')
        tempAnalysis.createSAEModels()
        tempSAE = tempAnalysis.getSAEModels()
        
        for inovelty in range(len(analysis.getClassLabels())):
            results[regularizer][regularizer_param][inovelty] = {}
            
            mse = {}
            mse_known = np.zeros([analysis.n_folds, len(neurons_mat)])
            mse_novelty = np.zeros([analysis.n_folds, len(neurons_mat)])
            
            for ineuron in neurons_mat:      
                results[regularizer][regularizer_param][inovelty][ineuron] = {}
                print("Regularization: {} {} Novelty: {} Neuron: {}".format(regularizer, regularizer_param, inovelty, ineuron))

                def processFold(ifold):
                    train_id, test_id = analysis.CVO[inovelty][ifold]        
                    saeModel = tempSAE[inovelty].getModel(analysis.trn_data[inovelty], analysis.trn_trgt[inovelty],
                                                hidden_neurons=[ineuron],
                                                layer=layer, ifold=ifold,
                                                regularizer=regularizer, regularizer_param=regularizer_param)             
                    
#                     classifier = tempSAE[inovelty].loadClassifier(analysis.trn_data[inovelty], analysis.trn_trgt[inovelty],
#                                                                   hidden_neurons=[ineuron],
#                                                                   layer=layer, ifold=ifold,
#                                                                   regularizer=regularizer, regularizer_param=regularizer_param)   
                    
                    # Normalize the data used based on the normalization used during the training              
                    knownData = tempSAE[inovelty].normalizeData(analysis.trn_data[inovelty][test_id, :], ifold)          
                    noveltyData = tempSAE[inovelty].normalizeData(all_data[all_trgt!=inovelty], ifold)

                    outputKnownData = saeModel.predict(knownData)
                    outputNoveltyData = saeModel.predict(noveltyData)
                    
                    
                    # Compute metrics
                    #################### Pre-training step #######################
                    # Mean squared error
                    mse_known = metrics.mean_squared_error(knownData, outputKnownData)
                    mse_novelty = metrics.mean_squared_error(noveltyData, outputNoveltyData)
                    
                    # Kullback-leibler divergence
                    
                    #################### Fine-tuning step ########################
                    
                    return ifold, mse_known, mse_novelty
                # Start Parallel processing
                p = multiprocessing.Pool(processes=num_processes)

                folds = range(analysis.n_folds)
                
                mse[ineuron] = p.map(processFold, folds)

                for ifold in range(analysis.n_folds):
                    mse_known[:, neurons_mat.index(ineuron)] = mse[ineuron][ifold][1]
                    mse_novelty[:, neurons_mat.index(ineuron)] = mse[ineuron][ifold][2]

                p.close()
                p.join()
                
                results[regularizer][regularizer_param][inovelty][ineuron]['mse_known'] = mse_known
                results[regularizer][regularizer_param][inovelty][ineuron]['mse_novelty'] = mse_novelty  
                
                    


Reading from /home/vinicius.mello/Workspace/SonarAnalysis/Results/NoveltyDetection/10_folds_cross_validation.jbl
Regularization: l1 0.1 Novelty: 0 Neuron: 1
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Regularization: l1 0.1 Novelty: 0 Neuron: 50
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Regularization: l1 0.1 Novelty: 0 Neuron: 75
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updati

In [14]:
from sklearn.externals import joblib

trn_params = joblib.load('/home/vinicius.mello/Workspace/SonarAnalysis/Results/NoveltyDetection/StackedAutoEncoder/Adam_optmizer/tanh_sae_hidden_activation/linear_sae_output_actvation/softmax_classifier_output_activation/2_init_10_folds_maptstd_norm_300_epochs_256_batch_size_accuracy_metric_mean_squared_error_loss/trnparams.jbl')
trn_params

[{'batch_size': 256,
  'beta_1': 0.9,
  'beta_2': 0.999,
  'classifier_output_actvation': 'softmax',
  'epsilon': 1e-08,
  'folds': 10,
  'hidden_activation': 'tanh',
  'learning_decay': 1e-06,
  'learning_rate': 0.001,
  'loss': 'mean_squared_error',
  'metrics': ['accuracy'],
  'momentum': 0.3,
  'n_epochs': 300,
  'n_inits': 2,
  'nesterov': True,
  'norm': 'maptstd',
  'optmizerAlgorithm': 'Adam',
  'output_activation': 'linear',
  'patience': 30,
  'train_verbose': False,
  'verbose': False}]

## Mean Squared Error analysis for Pre-training Step with a neuron variation at autoencoder

In [ ]:
# Neuron variation x KL Divergence
%matplotlib inline 

# Choose layer 
layer = 1

# Choose neurons topology
hidden_neurons = range(400,0,-50) + [2]

neurons_mat = [10, 20] + range(50,450,50)
neurons_mat = neurons_mat[:len(neurons_mat)-layer+2]

# Choose model
inovelty = 1
regularizer = "l2"
regularizer_param = 0.500000

# generate analysis data
save_path=results_path

current_analysis = 'mean_squared_error_%i_layer_%i_novelty'%(layer, inovelty)
analysis_str = 'StackedAutoEncoder'
model_prefix_str = 'RawData'

if regularizer != None or len(regularizer) != 0:
    analysis_file_name='%s/%s/%s_%s_regularizer(%f)_%s_neuron_number_sweep.jbl'%(results_path,analysis_str,analysis_name,regularizer,
                                                                                 regularizer_param,current_analysis)
else:
    analysis_file_name='%s/%s/%s_%s_neuron_number_sweep.jbl'%(results_path,analysis_str,analysis_name,current_analysis)

verbose = True

# Plot parameters
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handlelength'] = 3
plt.rcParams['legend.borderpad'] = 0.3
plt.rcParams['legend.fontsize'] = 18
m_colors = ['b', 'r', 'g', 'y']

params_str = analysis.trn_params.get_params_str()

n_folds = analysis.n_folds

mse = {}
mse_known = np.zeros([n_folds, len(neurons_mat)])
mse_novelty = np.zeros([n_folds, len(neurons_mat)])

# if os.path.exists(analysis_file_name):
#     os.remove(analysis_file_name)
    
if not os.path.exists(analysis_file_name):
    for ineuron in neurons_mat: 
        if ineuron == 0:
            ineuron = 1
        neurons_str = SAE[inovelty].getNeuronsString(all_data, hidden_neurons=hidden_neurons[:layer-1]+[ineuron])

        models = {}
        outputs = {}
        norm_data = {}
        reconstructed_known_data = {}
        reconstructed_novelty_data = {}
        if verbose: 
            print '[*] Layer: %i - Topology: %s'%(layer, neurons_str)

        n_bins = 100

        def getMSE(ifold):
            train_id, test_id = analysis.CVO[inovelty][ifold]

            # normalize known classes
            if trn_params.params['norm'] == 'mapstd':
                scaler = preprocessing.StandardScaler().fit(all_data[all_trgt!=inovelty][train_id,:])
            elif trn_params.params['norm'] == 'mapstd_rob':
                scaler = preprocessing.RobustScaler().fit(all_data[all_trgt!=inovelty][train_id,:])
            elif trn_params.params['norm'] == 'mapminmax':
                scaler = preprocessing.MinMaxScaler().fit(all_data[all_trgt!=inovelty][train_id,:])

            known_data = scaler.transform(all_data[all_trgt!=inovelty][test_id,:])
            novelty_data = scaler.transform(all_data[all_trgt==inovelty])

            model = SAE[inovelty].getModel(data=all_data, trgt=all_trgt,
                                           hidden_neurons=hidden_neurons[:layer-1]+[ineuron],
                                           layer=layer, ifold=ifold, regularizer=regularizer, regularizer_param=regularizer_param)

            known_output = model.predict(known_data)
            novelty_output = model.predict(novelty_data)
            
            mseKnown = metrics.mean_squared_error(known_data, known_output)
            mseNovelty = metrics.mean_squared_error(novelty_data, novelty_output)
            
                
            return ifold, mseKnown, mseNovelty

        # Start Parallel processing
        p = multiprocessing.Pool(processes=num_processes)

        folds = range(n_folds)
        if verbose:
            print '[*] Calculating Mean Squared Error ...'
        mse[ineuron] = p.map(getMSE, folds)
        
        for ifold in range(n_folds):
            mse_known[:, neurons_mat.index(ineuron)] = mse[ineuron][ifold][1]
            mse_novelty[:, neurons_mat.index(ineuron)] = mse[ineuron][ifold][2]
        
        p.close()
        p.join()
    
        joblib.dump([neurons_mat,mse_known,mse_novelty],analysis_file_name,compress=9)
else:
    [neurons_mat, mse_known, mse_novelty] = joblib.load(analysis_file_name)
# Plot results    
fig, m_ax = plt.subplots(figsize=(20,30),nrows=5, ncols=2)

for ifold in range(n_folds):
    irow = int(ifold/2)
    if (ifold % 2 == 0):
        icolumn = 0
    else: 
        icolumn = 1

    m_ax[irow, icolumn].plot(neurons_mat, mse_known[ifold, :], 'b-o', label='Known Test Data')
    m_ax[irow, icolumn].plot(neurons_mat, mse_novelty[ifold, :], 'r--o', label='Novelty Data')
    m_ax[irow, icolumn].set_title('MSE x Neurons - Layer %i - Fold %i - Novelty Class %i '%(layer,ifold+1, inovelty), fontsize=16, fontweight='bold')
    m_ax[irow, icolumn].set_ylabel('Mean Squared Error', fontsize=22)
    m_ax[irow, icolumn].set_xlabel('Neurons', fontsize=22)
    m_ax[irow, icolumn].grid()
    m_ax[irow, icolumn].legend()
    plt.tight_layout()
plt.show()
#Save the figure
file_name = pict_results_path+'/'+current_analysis+'_%i_novelty_%s_neurons_'%(inovelty,neurons_str)+trn_params.get_params_str()+'.pdf'
plt.savefig(file_name)
print trn_params.get_params_str()
print '%s neurons'%neurons_str

## Receiver Operating Characteristic (ROC) Curve for SP/Trigger with Novelty Detection

In [ ]:
from __future__ import unicode_literals

# Choose layer
layer = 1
inovelty = 1

# Choose neurons topology for SAE
hidden_neurons = range(400,0,-50) + [2]

neurons_str = SAE[inovelty].getNeuronsString(all_data, hidden_neurons=hidden_neurons[:layer])

# generate analysis data
save_path=results_path

analysis_str = 'StackedAutoEncoder'
model_prefix_str = 'RawData'
verbose = False

current_analysis = 'figures_of_merit'

analysis_file_name='%s/%s/%s_novelty_detection_%s_neurons_thr_sweep.jbl'%(results_path,analysis_str,analysis_name, neurons_str)

if not os.path.exists(analysis_file_name):

    params_str = trn_params.get_params_str()

    thr_mat = np.round(np.arange(-0.0,1.05,0.05),3)
    thr_mat[thr_mat>-0.1] = abs(thr_mat[thr_mat>-0.1])

    class_eff_mat = np.zeros([n_folds,len(np.unique(all_trgt)),len(thr_mat)])
    novelty_eff_mat = np.zeros([n_folds,len(thr_mat)])
    known_acc_mat = np.zeros([n_folds,len(thr_mat)])
    known_sp_mat = np.zeros([n_folds,len(thr_mat)])
    known_trig_mat = np.zeros([n_folds,len(thr_mat)])

    class_eff = np.zeros([len(np.unique(all_trgt))], dtype=object)
    novelty_eff = np.zeros([len(np.unique(all_trgt))], dtype=object)
    known_acc = np.zeros([len(np.unique(all_trgt))], dtype=object)
    known_sp = np.zeros([len(np.unique(all_trgt))], dtype=object)
    known_trig = np.zeros([len(np.unique(all_trgt))], dtype=object)

    def getFiguresMetrics(inovelty):
        n_folds = len(CVO[inovelty])  
        print 'Novelty class: %01.0f - topology: %s'%(inovelty, neurons_str)
        for ifold in range(n_folds):
            classifier = SAE[inovelty].loadClassifier(data  = trn_data[inovelty],
                                                      trgt  = trn_trgt[inovelty], 
                                                      hidden_neurons = hidden_neurons[:layer],
                                                      layer = layer,
                                                      ifold = ifold)

            train_id, test_id = CVO[inovelty][ifold]
            # normalize known classes
            if trn_params.params['norm'] == 'mapstd':
                scaler = preprocessing.StandardScaler().fit(trn_data[inovelty][train_id,:])
            elif trn_params.params['norm'] == 'mapstd_rob':
                scaler = preprocessing.RobustScaler().fit(trn_data[inovelty][train_id,:])
            elif trn_params.params['norm'] == 'mapminmax':
                scaler = preprocessing.MinMaxScaler().fit(trn_data[inovelty][train_id,:])

            known_data = scaler.transform(trn_data[inovelty][test_id,:])
#             known_data = trn_data[inovelty][test_id,:]
            known_trgt = trn_trgt[inovelty][test_id]

            novelty_data = scaler.transform(all_data[all_trgt==inovelty])
#             novelty_data = all_data[all_trgt==inovelty]

            output = classifier.predict(known_data)
            novelty_output = classifier.predict(novelty_data)

            for ithr,thr_value in enumerate(thr_mat): 
                buff = np.zeros([len(np.unique(all_trgt))-1])
                for iclass, class_id in enumerate(np.unique(all_trgt)):
                    if iclass == inovelty:
                        continue
                    output_of_class_events = output[known_trgt==iclass-(iclass>inovelty),:]
                    correct_class_output = np.argmax(output_of_class_events,axis=1)==iclass-(iclass>inovelty)
                    output_above_thr = output_of_class_events[correct_class_output,iclass-(iclass>inovelty)]>thr_value
                    class_eff_mat[ifold, iclass, ithr] = float(sum(output_above_thr))/float(len(output_of_class_events))
                    buff[iclass-(iclass>inovelty)] = class_eff_mat[ifold, iclass, ithr]
                novelty_eff_mat[ifold, ithr] = float(sum(1-(novelty_output>thr_value).any(axis=1)))/float(len(novelty_output))
                known_acc_mat[ifold, ithr] = np.mean(buff,axis=0)
                known_sp_mat[ifold, ithr]= (np.sqrt(np.mean(buff,axis=0)
                                                              *np.power(np.prod(buff),1./float(len(buff)))))
                known_trig_mat[ifold, ithr]=float(sum(np.max(output,axis=1)>thr_value))/float(len(output))

        return inovelty, class_eff_mat, novelty_eff_mat, known_acc_mat, known_sp_mat, known_trig_mat

    # Start Parallel processing
    p = multiprocessing.Pool(processes=num_processes)

    results = p.map(getFiguresMetrics, class_labels.keys())

    p.close()
    p.join()         

    for inovelty in class_labels.keys():
        if inovelty == results[inovelty][0]:
            class_eff[inovelty]   = results[inovelty][1]
            novelty_eff[inovelty] = results[inovelty][2]
            known_acc[inovelty]   = results[inovelty][3]
            known_sp[inovelty]    = results[inovelty][4]
            known_trig[inovelty]  = results[inovelty][5]

    joblib.dump([class_eff, novelty_eff, known_acc, known_sp, known_trig, thr_mat],
                analysis_file_name,compress=9)
else:
    print 'File exists'
    [class_eff, novelty_eff, known_acc, known_sp, known_trig, thr_mat] = joblib.load(analysis_file_name) 

# plot analysis
import matplotlib.pyplot as plt
%matplotlib inline  


plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['legend.numpoints'] = 1
plt.rc('legend',**{'fontsize':15})
plt.rc('font', weight='bold')

m_colors = ['b', 'r', 'g', 'y']

for inovelty, novelty_class in enumerate(np.unique(all_trgt)):
    fig = plt.subplots(figsize=(12,6))
    ax = plt.subplot(1,1,1)
    
    xdata1 = np.ones(np.mean(known_sp[inovelty], axis=0).shape) - np.mean(known_sp[inovelty], axis=0)
    xdata2 = np.ones(np.mean(known_sp[inovelty], axis=0).shape) - np.mean(known_trig[inovelty], axis=0)
        
    ax.errorbar(xdata1, np.mean(novelty_eff[inovelty], axis=0),
                np.std(novelty_eff[inovelty], axis=0),fmt='o-',
                color='r',alpha=0.7,linewidth=2.5,
                label='ROC SP')
    ax.errorbar(xdata2, np.mean(novelty_eff[inovelty], axis=0),
                np.std(novelty_eff[inovelty], axis=0),fmt='d-',
                color='b',alpha=0.7,linewidth=2.5,
                label='ROC Trigger')
#     ax.errorbar(thr_mat,np.mean(known_acc[inovelty], axis=0),
#                 np.std(known_acc[inovelty], axis=0),fmt='o--',
#                 color='k',alpha=0.7,linewidth=2.5,
#                 label='Known Acc.')
    ax.set_xticks(np.arange(0,1.25,0.25))
    ax.set_xticklabels(np.arange(0,1.25,0.25),rotation=45, fontsize=18)
    ax.set_title('SAE com %i camada treinado com a classe %s como Novidade'%(layer,class_labels_txt[inovelty]),fontsize=18,weight='bold')
    ax.set_xlim([np.min(thr_mat), np.max(thr_mat)])
    
    ax.set_ylim([0.0, 1.2])
    y_ticks = np.arange(0.0,1.3,0.1)
    ax.set_yticks(np.round(y_ticks,2))
    ax.set_yticklabels(100*np.round(y_ticks,2)[np.round(y_ticks,2)<=1.0],fontsize=18)
    
    ax.grid()
    
    ax.set_xlabel('(1 - SP) (%s)',fontsize=20,weight='bold')
    ax.set_ylabel('Detecção de Novidade (%)',fontsize=20,weight='bold')

    handles, labels = ax.get_legend_handles_labels()
    # sort both labels and handles by labels
    labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
    ax.legend(handles, labels, ncol=3, loc='upper center')
    
    plt.show()
    #Save the figure
    file_name = pict_results_path+'/'+"ROC_sp_novelty_det"+'_%i_novelty_%s_neurons_'%(inovelty,neurons_str)+trn_params.get_params_str()+'.pdf'
    plt.savefig(file_name)
print trn_params.get_params_str()
print '%s neurons'%neurons_str